In [1]:
import os
import re
import datetime
import random

In [2]:
datafolder='./midi_audio'

In [3]:
outputfolder='./midi_audio_sample' + datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')
outputfolder_testing=outputfolder + '/testing'
outputfolder_teacher=outputfolder + '/teacher'

print('creating directory: ', outputfolder_testing, outputfolder_teacher)

os.makedirs(outputfolder)
os.makedirs(outputfolder_testing)
os.makedirs(outputfolder_teacher)

creating directory:  ./midi_audio_sample20180819013100/testing ./midi_audio_sample20180819013100/teacher


In [4]:
files = os.listdir(datafolder)

In [5]:
csv_source={}

In [6]:
for file in files:
    m = re.match('^note-program(?P<program>\d+)-degree(?P<degree>\d+)-volume(?P<volume>\d+).*.ogg$', file)
    v_program = m.group('program')
    v_degree = m.group('degree')
    v_volume = m.group('volume')
    
    if int(v_program) >= 10:
        continue # limit programs
    
    is_testing_data = random.random() < 0.2
    
    if is_testing_data:
        f_outputfolder = outputfolder_testing
    else:
        f_outputfolder = outputfolder_teacher
    f_outputfile = f_outputfolder + '/' + file
    
    #print('creating link: ', file, ' ... ', f_outputfile)
    file_fullpath = '../../../notebooks/midi_audio/' + file #os.path.abspath('.') + '/' + datafolder + '/' + file
    os.symlink(file_fullpath, f_outputfile)
    
    if is_testing_data not in csv_source:
        csv_source[is_testing_data] = {}
    if v_program not in csv_source[is_testing_data]:
        csv_source[is_testing_data][v_program] = {}
    csv_source[is_testing_data][v_program][file] = 1.0
    

In [7]:
#csv_source
print(len(csv_source[True]))
print(len(csv_source[False]))

10
10


In [8]:
csv_fr = {} #file and rows

In [9]:
for is_testing_data in csv_source.keys():
    dict = csv_source[is_testing_data]
    programs = list(dict.keys())
    files = [item for sublist in [list(dict[p]) for p in programs] for item in sublist]
    combinations = [(program, file) for program in programs for file in files]
    csv_fr[is_testing_data] = {}
    for combination in combinations:
        program = combination[0]
        file = combination[1]
        if program not in csv_fr[is_testing_data]:
            csv_fr[is_testing_data][program] = []
        if program not in dict or file not in dict[program]:
            csv_fr[is_testing_data][program].append([file, 0.0])
        else:
            csv_fr[is_testing_data][program].append([file, dict[program][file]])

In [10]:
#csv_fr

In [11]:
for is_testing_data in csv_fr:
    if is_testing_data:
        folder = outputfolder_testing
    else:
        folder = outputfolder_teacher
    for program in csv_fr[is_testing_data]:
        csvfile = folder + '/' + 'program{0}.csv'.format(program)
        with open(csvfile, 'w') as f:
            for row in csv_fr[is_testing_data][program]:
                audio_file = row[0]
                value = row[1]
                f.write('{0},{1}\r\n'.format(audio_file, value))